In [2]:
import pandas as pd

Here we load the dataset

In [3]:
df = pd.read_csv('/Users/alihamzeh/Downloads/genai_case/companies_data.csv')
df

,companyName,description
0,Traton SE,Traton SE manufactures commercial vehicles wor...
1,2G Energy AG,"2G Energy AG, together with its subsidiaries, ..."
2,MTU Aero Engines AG,"MTU Aero Engines AG, together with its subsidi..."
3,Deutsche Lufthansa AG,Deutsche Lufthansa AG operates as an aviation ...
4,Siemens Energy AG,Siemens Energy AG operates as an energy techno...
...,...,...
185,RATIONAL Aktiengesellschaft,RATIONAL Aktiengesellschaft engages in the dev...
186,Vossloh AG,Vossloh AG manufactures and markets rail infra...
187,Singulus Technologies AG,"Singulus Technologies AG, together with its su..."
188,KUKA Aktiengesellschaft,"KUKA Aktiengesellschaft, an automation company..."


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 190 entries, 0 to 189
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   companyName  190 non-null    object
 1   description  190 non-null    object
dtypes: object(2)
memory usage: 3.1+ KB


In [5]:
df.describe()

,companyName,description
count,190,190
unique,105,110
top,KUKA Aktiengesellschaft,"KUKA Aktiengesellschaft, an automation company..."
freq,5,4


As observed, there are a total of 190 company entries, but only 105 unique rows. Therefore, we need to examine whether these entries are true duplicates or whether they contain differing information.

In [14]:
# 1. Group by companyName and count the number of unique descriptions for each company
unique_descriptions_per_company = df.groupby('companyName')['description'].nunique().reset_index()
unique_descriptions_per_company.columns = ['companyName', 'unique_description_count']

# 2. Filter for companies where the name is duplicated (count > 1) AND the descriptions are NOT identical (unique_description_count > 1).
# We also want to check for companies where the name is duplicated, so we first find all duplicated company names.
duplicated_names = df['companyName'].value_counts()
duplicated_names = duplicated_names[duplicated_names > 1].index

# Filter the unique_descriptions_per_company to only include companies with duplicated names
companies_to_check = unique_descriptions_per_company[unique_descriptions_per_company['companyName'].isin(duplicated_names)]

# Identify companies that have a repeated name but different descriptions
companies_with_different_descriptions = companies_to_check[companies_to_check['unique_description_count'] > 1]

# Identify companies that have a repeated name and the SAME description
companies_with_same_descriptions = companies_to_check[companies_to_check['unique_description_count'] == 1]

print("--- 1. Companies with Duplicated Names but DIFFERENT Descriptions (Problematic for RAG) ---")
if not companies_with_different_descriptions.empty:
    print(companies_with_different_descriptions.to_markdown(index=False, numalign="left", stralign="left"))
else:
    print("No companies found with duplicated names but different descriptions.")

print("\n--- 2. Companies with Duplicated Names and the SAME Description ---")
if not companies_with_same_descriptions.empty:
    print(companies_with_same_descriptions.to_markdown(index=False, numalign="left", stralign="left"))
else:
    print("No companies found with duplicated names and the same description.")

print("\n--- 3. Inspecting a Sample of Duplicates with the SAME Description ---")
# Select the first 3 problematic companies to show their original data (if any exist)
sample_companies = companies_with_same_descriptions['companyName'].head(3).tolist()

if sample_companies:
    sample_df = df[df['companyName'].isin(sample_companies)]
    print(sample_df.sort_values(by='companyName').to_markdown(index=False, numalign="left", stralign="left"))
else:
    print("No sample data to display as there are no duplicates with the same description.")

--- 1. Companies with Duplicated Names but DIFFERENT Descriptions (Problematic for RAG) ---
| companyName             | unique_description_count   |
|:------------------------|:---------------------------|
| KUKA Aktiengesellschaft | 2                          |
| MAN SE                  | 2                          |
| SLM Solutions Group AG  | 2                          |
| Sixt SE                 | 2                          |
| thyssenkrupp AG         | 2                          |

--- 2. Companies with Duplicated Names and the SAME Description ---
| companyName                                     | unique_description_count   |
|:------------------------------------------------|:---------------------------|
| Allane SE                                       | 1                          |
| Aumann AG                                       | 1                          |
| BAUER Aktiengesellschaft                        | 1                          |
| BayWa Aktiengesellschaft         

First, we remove redundant rows that are exact duplicates.

In [15]:
df = df.drop_duplicates(subset=['companyName'], keep='first').reset_index(drop=True)
df.describe()

,companyName,description
count,105,105
unique,105,105
top,Traton SE,Traton SE manufactures commercial vehicles wor...
freq,1,1


Assuming that all information in the dataset is correct and that the second version of each company description contains additional details, we merge the descriptions using an LLM manually.

In [19]:

# 1. Update KUKA Aktiengesellschaft with the merged description
# This updates all rows matching the companyName
df.loc[df["companyName"] == "KUKA Aktiengesellschaft", "description"] = (
    "KUKA Aktiengesellschaft, an automation company, provides robot-based automation solutions worldwide. It operates through five segments: Systems, Robotics, Swisslog, Swisslog Healthcare, and China. The company manufactures and supplies industrial, collaborative, and mobile robots, as well as robot controllers, software, and digital services for industrial Internet of Things. It also offers automated guided vehicles and other automation components to production cells, turnkey systems, and networked production with the aid of cloud-based IT tools; individual system components, tools and fixtures, and automated production cells; and robot-based and modular manufacturing cells, as well as support services. In addition, the company offers automated solutions for hospitals, warehouses, and distribution centers; and warehouse management systems and healthcare systems. It serves customers in the automotive, electronics, e-commerce/retail, consumer goods, metal and plastic, healthcare, and other industries. The company was formerly known as Industrie-Werke Karlsruhe Augsburg Aktiengesellschaft and changed its name to KUKA Aktiengesellschaft in 2007. The company was founded in 1898 and is headquartered in Augsburg, Germany. KUKA Aktiengesellschaft is a subsidiary of Midea Electric Netherlands (I) B.V. KUKA Aktiengesellschaft is a subsidiary of GD Midea Holding Co., Ltd."
)

# 2. Update MAN SE with the merged description
df.loc[df["companyName"] == "MAN SE", "description"] = (
    "MAN SE operates in commercial vehicle industry in Germany and internationally. It operates through two segments, MAN Truck & Bus and MAN Latin America. The MAN Truck & Bus segment offers commercial vehicles. This segment provides vans, trucks, buses, and diesel and gas engines, as well as passenger and freight transportation services. The MAN Latin America segment manufactures and sells trucks and buses primarily under the Volkswagen Caminhões e Ônibus brand. The company was founded in 1758 and is headquartered in Munich, Germany. MAN SE operates as a subsidiary of Traton SE. It operates through two divisions, MAN Truck & Bus and MAN Latin America. The MAN Truck & Bus division manufactures commercial vehicles. This division provides vans, trucks, buses, and diesel and gas engines, as well as passenger and freight transportation services. The MAN Latin America division manufactures and sells trucks and buses primarily under the Volkswagen Caminhões e Ônibus brand."
)

# 3. Update SLM Solutions Group AG with the merged description
df.loc[df["companyName"] == "SLM Solutions Group AG", "description"] = (
    "SLM Solutions Group AG provides metal-based additive manufacturing technology solutions in Germany, the Asia/Pacific, other European countries, North America, and internationally. The company operates through two segments, Machine Business and After Sales Business. The Machine Business segment engages in the development, production, marketing, and sale of machines and peripheral equipment for selective laser melting. The After Sales Business segment provides machine-related and other services; and sells replacement parts, accessories, merchandise, and consumables. The company also offers software, metal powder, quality assurance products, and consulting services. It serves aerospace and defense, automotive, energy, healthcare, tooling, and research industries. The company is headquartered in Lübeck, Germany. SLM Solutions Group AG operates as subsidiary of Nikon AM. AG."
)

# 4. Update Sixt SE with the merged description
df.loc[df["companyName"] == "Sixt SE", "description"] = (
    "Sixt SE, through its subsidiaries, provides mobility services for private and business customers in Germany and internationally. The company engages in the rental of various utility vehicles; and provision of international holiday vehicle rental, car sharing, transfer, and car subscription services. As of December 31, 2021, its corporate and franchise station network comprised approximately 900 stations. The company was founded in 1912 and is headquartered in Pullach, Germany. Sixt SE is a subsidiary of Erich Sixt Vermögensverwaltung GmbH. The company offers mobility service across the fields of vehicle and commercial vehicle rental, car sharing, ride hailing, and car subscriptions. It has a network of 2,115 stations."
)

# 5. Update thyssenkrupp AG with the merged description
df.loc[df["companyName"] == "thyssenkrupp AG", "description"] = (
    "thyssenkrupp AG operates in the areas of automotive technology, industrial components, marine systems, steel, and materials services in Germany, the United States, China, and internationally. The company's Automotive Technology segment develops and manufactures components and systems, as well as automation solutions for the automotive industry. Its Industrial Components segment manufactures and sells forged components and system solutions for the resource, construction, and mobility sectors; and slewing rings, antifriction bearings, and seamless rolled rings for the wind energy and construction machinery sectors. The company's Multi Tracks segment builds plants for the chemical, cement, and mining industries. Its Marine Systems segment provides systems in the submarine and surface vessel construction, as well as in the field of maritime electronics and security technology. The company's Materials Services segment distributes materials and offers technical services for the production and manufacturing sectors. Its Steel Europe segment provides flat carbon steel products, intelligent material solutions, and finished parts. thyssenkrupp AG was founded in 1811 and is headquartered in Essen, Germany. thyssenkrupp AG operates in the areas of materials services, industrial components, automotive technology, steel, and marine systems in Germany, the United States, China, and internationally. The company's Automotive Technology segment develops and manufactures components and systems, as well as automation solutions for the vehicle manufacturing. The company's Marine Systems segment offers systems in the submarine and surface vessel construction, as well as in the field of maritime electronics and security technology."
)

In [27]:
print(df[df["companyName"] == "KUKA Aktiengesellschaft"])

                companyName                                        description
89  KUKA Aktiengesellschaft  KUKA Aktiengesellschaft, an automation company...


Furthermore, we examine the length of each document to determine the number of words it contains, which informs the subsequent document-splitting strategy.

In [28]:
df["description_word_count"] = (
    df["description"]
    .astype(str)
    .str.split()
    .str.len()
)


In [29]:
df.describe()

,description_word_count
count,105.000000
mean,158.342857
std,56.120206
min,41.000000
25%,115.000000
50%,158.000000
75%,201.000000
max,275.000000


In [ ]:
# Save the cleaned dataset
df.to_csv('cleaned_companies_data.csv', index=False)

To make the RAG system more generic and robust, I use DeepSeek to enrich the company descriptions with additional information. Since the performance of the RAG pipeline is highly dependent on the underlying text, this step expands the original content with more descriptive language, relevant synonyms, and contextual details, thereby improving retrieval quality.